In [ ]:
lines = open('input1.txt', 'r')
dial = 50
number_of_zeros = 0
number_of_clicks = 0

for line in lines:
	rotation = int(line[1:]) * (-1 if line[0] == 'L' else 1)
	dial %= (100 if rotation > 0 else -100)

	dial += rotation

	number_of_clicks += abs(dial) // 100
	number_of_zeros += 1 if dial % 100 == 0 else 0

print(f'The number of times the dial stops at zero is {number_of_zeros}')
print(f'The number of times the dial touches zero is {number_of_clicks}')

In [ ]:
from collections import defaultdict
import math

ranges_string = '11-22,95-115,998-1012,1188511880-1188511890,222220-222224,1698522-1698528,446443-446449,38593856-38593862,565653-565659,824824821-824824827,2121212118-2121212124'
ranges_string = open('input2.txt', 'r').read()
ranges = []
for id_range in ranges_string.split(','):
	start, end = id_range.strip().split('-')
	ranges.append((int(start), int(end)))

invalid_ids = defaultdict(lambda: set())

for start, end in ranges:
	min_digits = len(str(start))
	max_digits = len(str(end))

	for repeats in range(2, max_digits+1):
		min_guess_digits = math.ceil(min_digits / repeats)
		max_guess_digits = math.floor(max_digits / repeats)

		for number_of_digits in range(min_guess_digits, max_guess_digits+1):
			start_range = int(str(start)[:number_of_digits])
			end_range = int(str(end)[:number_of_digits])

			guess = int(start_range)
			while True:
				full_number = int(str(guess) * repeats)
				if start <= full_number <= end:
					invalid_ids[repeats].add(full_number)

				if guess == end_range:
					break
					
				guess = (guess + 1) % 10 ** (number_of_digits)

print(f'The sum of all invalid IDs that repeat twice is {sum(invalid_ids[2])}')
all_invalid_ids = set()
for repeats in invalid_ids:
	all_invalid_ids = all_invalid_ids.union(invalid_ids[repeats])
print(f'The sum of all invalid IDs is {sum(all_invalid_ids)}')

In [ ]:
import time

before = time.time()
joltages = open('input3.txt', 'r').read().strip().split('\n')

def find_highest_joltages(joltages, number_of_digits):
	highest = []
	for line in joltages:
		digits = [int(c) for c in line.strip()]
		found_digits = []
		starting_index = 0
		for i in range(number_of_digits-1, -1, -1):
			current_range = digits[starting_index:len(digits)-i]
			current_digit = max(current_range)
			found_digits.append(current_digit)
			starting_index += current_range.index(current_digit) + 1
		result = sum(digit * (10 ** index) for index, digit in enumerate(found_digits[::-1]))
		highest.append(result)
	return highest
	
highest = find_highest_joltages(joltages, 2)		
print(f'The sum of the highest joltages when picking two batteries is {sum(highest)}')

highest = find_highest_joltages(joltages, 12)
print(f'The sum of the highest joltages when picking twelve batteries is {sum(highest)}')
after = time

In [ ]:
paper = '''..@@.@@@@.
@@@.@.@.@@
@@@@@.@.@@
@.@@@@..@.
@@.@@@@.@@
.@@@@@@@.@
.@.@.@.@@@
@.@@@.@@@@
.@@@@@@@@.
@.@.@@@.@.'''.split('\n')
paper = open('input4.txt', 'r').read().strip().split('\n')

paper_positions = set([(x, y) for y, line in enumerate(paper) for x, symbol in enumerate(line) if symbol == '@'])

adjacent = [(1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0), (-1, -1), (0, -1), (1, -1)]

def check_surroundings(position):
	x, y = position
	surrounding_rolls = 0
	for dx, dy in adjacent:
		if (x + dx, y + dy) in paper_positions:
			surrounding_rolls += 1
	return surrounding_rolls

print(f'There are {sum(1 for position in paper_positions if check_surroundings(position) < 4)} accessible rolls of paper')

removed = set()
removables_exist = True

while removables_exist:
	removables = [position for position in paper_positions if check_surroundings(position) < 4]
	if len(removables) == 0:
		removables_exist = False
		break

	for removable in removables:
		removed.add(removable)
		paper_positions.remove(removable)

print(f'There are {len(removed)} rolls of paper that can be removed')

In [ ]:
fresh_ranges_input, ingredients_input = '''3-5
10-14
16-20
12-18

1
5
8
11
17
32'''.split('\n\n')
fresh_ranges_input, ingredients_input = open('input5.txt', 'r').read().strip().split('\n\n')

# Add ranges, while combining overlapping ones
def check_if_ranges_overlap(range_one, range_two):
	l1, r1 = range_one
	l2, r2 = range_two

	if l1 <= l2 <= r1 or l1 <= r2 <= r1 or l2 <= l1 <= r2:
		return True
	else:
		return False

fresh_ranges = set()
for fresh_range_input in fresh_ranges_input.split('\n'):
	current_range = tuple(int(x) for x in fresh_range_input.split('-'))

	while True:
		found_overlapping = False
		for other_range in fresh_ranges:
			if check_if_ranges_overlap(current_range, other_range):
				fresh_ranges.remove(other_range)
				current_range = (min(current_range[0], other_range[0]), max(current_range[1], other_range[1]))
				found_overlapping = True
				break

		if not found_overlapping:
			break

	fresh_ranges.add(current_range)

# Combine adjacent ranges
fresh_ranges = list(sorted(fresh_ranges, key=lambda t:t[0]))
counter = 0
while counter < len(fresh_ranges) - 1:
	current_range = fresh_ranges[counter]
	next_range = fresh_ranges[counter+1]

	if current_range[1] + 1 == next_range[0]:
		del fresh_ranges[counter+1]
		del fresh_ranges[counter]

		fresh_ranges.insert(counter, (current_range[0], next_range[1]))
	else:
		counter += 1

# Next, check the ingredients against the fresh ranges
def check_if_number_in_any_range(number, ranges):
	for start, end in ranges:
		if start <= number <= end:
			return True
	return False

ingredients = [int(x) for x in ingredients_input.split('\n')]
fresh_ingredients = [ingredient for ingredient in ingredients if check_if_number_in_any_range(ingredient, fresh_ranges)]
print(f'There are {len(fresh_ingredients)} fresh ingredients')
print(f'There are {sum(end - start + 1 for start, end in fresh_ranges)} fresh ingredient IDs')

In [ ]:
from collections import defaultdict
import re

maths_input = '''123 328  51 64 
 45 64  387 23 
  6 98  215 314
*   +   *   +'''.strip().split('\n')
maths_input = open('input6.txt', 'r').read().strip().split('\n')

columns = defaultdict(lambda: list())
for line in maths_input:
	line_columns = re.split(' +', line.strip())
	for index, line_column in enumerate(line_columns):
		columns[index].append(line_column)

answers = {}
for column in columns:
	column_array = columns[column]
	if column_array[-1] == '+':
		answer = sum(int(x) for x in column_array[:-1])
		answers[column] = answer
	else:
		product = 1
		for factor in column_array[:-1]:
			product *= int(factor)
		answers[column] = product

print(f'The sum of the column answers is {sum(answers[column] for column in answers)}')

next_answer = None
operator = None

max_length = max(len(line) for line in maths_input)
answers = []

for column in range(max_length):
	number_string = ''
	for line in maths_input:
		if column < len(line) and line[column] != ' ':
			if line[column] == '+' or line[column] == '*':
				if next_answer is not None:
					answers.append(next_answer)

				operator = line[column]
				if line[column] == '+':
					next_answer = 0
				else:
					next_answer = 1
			else:
				number_string += line[column]
	if number_string != '':
		if operator == '+':
			next_answer += int(number_string)
		elif operator == '*':
			next_answer *= int(number_string)

answers.append(next_answer)
print(f'The sum of the answers using cephalopod maths is {sum(answers)}')

In [ ]:
from collections import defaultdict

tachyon_beams = '''.......S.......
...............
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............'''.split('\n')
tachyon_beams = open('input7.txt', 'r').read().split('\n')

start = tachyon_beams[0].index('S')
splitters = []
for line in tachyon_beams:
	this_line = []
	for index, character in enumerate(line):
		if character == '^':
			this_line.append(index)
	if this_line:
		splitters.append(this_line)

beams = {start: 1}
splitting_events = 0

for splitter_line in splitters:
	new_beams = defaultdict(lambda: int(0))
	for beam in beams:
		if beam in splitter_line:
			new_beams[beam-1] += beams[beam]
			new_beams[beam+1] += beams[beam]
			splitting_events += 1
		else:
			new_beams[beam] += beams[beam]
	beams = new_beams
print(f'The beam is split {splitting_events} times')

number_of_timelines = sum(beams[beam] for beam in beams)
print(f'There are {number_of_timelines} different timelines at the end')

In [ ]:
import math
from collections import defaultdict

junction_boxes_input = '''162,817,812
57,618,57
906,360,560
592,479,940
352,342,300
466,668,158
542,29,236
431,825,988
739,650,466
52,470,668
216,146,977
819,987,18
117,168,530
805,96,715
346,949,466
970,615,88
941,993,340
862,61,35
984,92,344
425,690,689'''.split('\n')
junction_boxes_input = open('input8.txt', 'r').read().strip().split('\n')
pairs_to_connect = 1000

junction_boxes = []
for line in junction_boxes_input:
	x, y, z = line.split(',')
	junction_boxes.append((int(x), int(y), int(z)))
junction_boxes.sort(key=lambda t:t[0])

distances = {}
for i in range(len(junction_boxes)):
	start = junction_boxes[i]
	distances[start] = {}
	for j in range(i+1, len(junction_boxes)):
		end = junction_boxes[j]
		distance = math.sqrt((end[0]-start[0])**2 + (end[1]-start[1])**2 + (end[2]-start[2])**2)
		distances[start][end] = distance

circuit_counter = 0
circuits = {}
connections = defaultdict(lambda: list())

distances_array = [(start, end, distances[start][end]) for start in distances for end in distances[start]]
distances_array.sort(key=lambda t:t[2])

def connect_pair(start, end, circuit_counter):
	connections[start].append(end)
	connections[end].append(start)
	
	if not start in circuits and not end in circuits:
		circuits[start] = circuit_counter
		circuits[end] = circuit_counter
		circuit_counter += 1

	elif start in circuits and not end in circuits:
		circuits[end] = circuits[start]
	elif end in circuits and not start in circuits:
		circuits[start] = circuits[end]

	elif circuits[start] != circuits[end]:
		new_circuit_number = circuits[start]

		nodes_to_change = [end]
		nodes_to_change_counter = 0
		while nodes_to_change_counter < len(nodes_to_change):
			current_node = nodes_to_change[nodes_to_change_counter]
			for connected_node in connections[current_node]:
				if circuits[connected_node] != new_circuit_number:
					nodes_to_change.append(connected_node)
			circuits[current_node] = new_circuit_number

			nodes_to_change_counter += 1

	return circuit_counter


for i in range(pairs_to_connect):
	start, end, _ = distances_array[i]
	circuit_counter = connect_pair(start, end, circuit_counter)

grouped_circuits = defaultdict(lambda: list())
for node in circuits:
	circuit_number = circuits[node]
	grouped_circuits[circuit_number].append(node)

grouped_circuits = sorted([grouped_circuits[circuit_number] for circuit_number in grouped_circuits], reverse=True, key=lambda l:len(l))

product = 1
for circuit in grouped_circuits[:3]:
	product *= len(circuit)
print(f'The three largest circuit sizes multiplied together is {product}')

def check_if_all_connected():
	if len(circuits) < len(junction_boxes_input):
		return False

	mega_circuit = None
	for node in circuits:
		if mega_circuit is None:
			mega_circuit = circuits[node]

		if circuits[node] != mega_circuit:
			return False

	return True

pair_to_connect = pairs_to_connect
last_connected_pair = None
while not check_if_all_connected():
	start, end, _ = distances_array[pair_to_connect]
	circuit_counter = connect_pair(start, end, circuit_counter)
	last_connected_pair = start, end
	pair_to_connect += 1

print(f'The product of the x coordinates of the last two connected junction boxes is {last_connected_pair[0][0] * last_connected_pair[1][0]}')

In [ ]:
red_tiles_input = '''7,1
11,1
11,7
9,7
9,5
2,5
2,3
7,3'''.split('\n')

red_tiles_input = open('input9.txt', 'r').read().strip().split('\n')

red_tiles = []
for line in red_tiles_input:
	x, y = [int(n) for n in line.split(',')]
	red_tiles.append((x, y))

largest_area = 0
best_tiles = None
for i in range(len(red_tiles)):
	sx, sy = red_tiles[i]
	for j in range(i+1, len(red_tiles)):
		ex, ey = red_tiles[j]
		area = (abs(ex - sx) + 1) * (abs(ey - sy) + 1)
		if area > largest_area:
			largest_area = area
			best_tiles = ((sx, sy), (ex, ey))
print(f'The largest area possible between red tiles is {largest_area}')

# Check if polygon is counter-clockwise, by checking if the shoelace algorithm gives a positive result
area_of_polygon = 0
for i in range(len(red_tiles)):
	cx, cy = red_tiles[i]
	px, py = red_tiles[i - 1]

	area_of_polygon += px * cy - cx * py

if area_of_polygon < 0:
	red_tiles = red_tiles[::-1]

def get_direction(start, end):
	sx, sy = start
	ex, ey = end

	if sy == ey and ex > sx:
		return 'right'
	if sy == ey and ex < sx:
		return 'left'
	if sx == ex and ey > sy:
		return 'down'
	if sx == ex and ey < sy:
		return 'up'
	else:
		return 'Error'

polygon = []
for i in range(len(red_tiles)):
	before = red_tiles[(i - 1) % len(red_tiles)]
	current = red_tiles[i]
	after = red_tiles[(i + 1) % len(red_tiles)]

	direction_before = get_direction(before, current)
	direction_after = get_direction(current, after)

	cx, cy = current
	if direction_after == 'right':
		if direction_before == 'right':
			polygon.append((cx, cy-0.5))
		elif direction_before == 'down':
			polygon.append((cx + 0.5, cy - 0.5))
		elif direction_before == 'up':
			polygon.append((cx - 0.5, cy - 0.5))

	elif direction_after == 'down':
		if direction_before == 'down':
			polygon.append((cx + 0.5, cy))
		elif direction_before == 'left':
			polygon.append((cx + 0.5, cy + 0.5))
		elif direction_before == 'right':
			polygon.append((cx + 0.5, cy - 0.5))

	elif direction_after == 'left':
		if direction_before == 'left':
			polygon.append((cx, cy + 0.5))
		elif direction_before == 'down':
			polygon.append((cx + 0.5, cy + 0.5))
		elif direction_before == 'up':
			polygon.append((cx - 0.5, cy + 0.5))

	elif direction_after == 'up':
		if direction_before == 'up':
			polygon.append((cx - 0.5, cy))
		elif direction_before == 'left':
			polygon.append((cx - 0.5, cy + 0.5))
		elif direction_before == 'right':
			polygon.append((cx - 0.5, cy - 0.5))

def check_if_lines_cross(start_one, end_one, start_two, end_two):
	x1, y1 = start_one
	x2, y2 = end_one
	x3, y3 = start_two
	x4, y4 = end_two

	divisor = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
	if divisor == 0:
		return False

	length_along_line_one = ((x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)) / divisor
	if not 0 <= length_along_line_one <= 1:
		return False

	length_along_line_two = -((x1 - x2) * (y1 - y3) - (y1 - y2) * (x1 - x3)) / divisor
	if not 0 <= length_along_line_two <= 1:
		return False

	return True
	

# Do a new check while also checking all rectangle lines stay within the polygon
largest_area = 0
illegal_lines = set()
legal_lines = set()
for i in range(len(red_tiles)):
	start_node = red_tiles[i]
	sx, sy = start_node
	for j in range(i+1, len(red_tiles)):
		end_node = red_tiles[j]
		ex, ey = end_node

		area = (abs(sx - ex) + 1) * (abs(sy - ey) + 1)
		if area > largest_area:
			# Check if rectangle is entirely inside polygon
			is_inside = True
			lines = [
				((min(sx, ex), min(sy, ey)), (min(sx, ex), max(sy, ey))),
				((min(sx, ex), max(sy, ey)), (max(sx, ex), max(sy, ey))),
				((max(sx, ex), max(sy, ey)), (max(sx, ex), min(sy, ey))),
				((max(sx, ex), min(sy, ey)), (min(sx, ey), min(sy, ey)))
			]

			for line in lines:
				if line in illegal_lines:
					is_inside = False
				elif line in legal_lines:
					continue
					
				if not is_inside:
					break

				line_start, line_end = line
				for k in range(len(polygon)):
					polygon_start = polygon[k-1]
					polygon_end = polygon[k]

					if check_if_lines_cross(line_start, line_end, polygon_start, polygon_end):
						is_inside = False
						illegal_lines.add(line)
						break

				legal_lines.add(line)

			if is_inside:
				largest_area = area

print(f'The largest possible area inside the colored tiles is {largest_area}')

In [ ]:
%run ../Common/CommonMethods.ipynb
import re
import sympy

input_schematics = '''[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}'''.split('\n')
input_schematics = open('input10.txt', 'r').read().strip().split('\n')

machines = []
for line in input_schematics:
	lights = tuple(True if c == '#' else False for c in re.search(r'\[(.+)\]', line).group(1))
	buttons = [tuple(int(c) for c in button[1:-1].split(',')) for button in re.search(r'\(.+\)', line).group(0).split(' ')]
	joltages = tuple(int(c) for c in re.search(r'\{(.+)\}', line).group(1).split(','))
	machines.append((lights, buttons, joltages))

# Use Dijkstra for part one of problem
fewest_presses = []
for lights, buttons, _ in machines:
	initial_state = tuple(False for x in lights)
	
	def get_neighbours(state):
		neighbours = []
		for button in buttons:
			neighbours.append(tuple(not light if index in button else light for index, light in enumerate(state)))
		return neighbours
		
	fewest_presses.append(perform_a_star(initial_state, lights, get_neighbours)[0][-1][1])
	
print(f'The fewest possible presses to turn on all the lights is {sum(fewest_presses)}')

def recursive_minimize(buttons, joltages):
	# Set up augmented matrix
	matrix = sympy.zeros(len(joltages), len(buttons)+1)
	for column, button in enumerate(buttons):
		for row in button:
			matrix[row, column] = 1
	for row, joltage in enumerate(joltages):
		matrix[row, len(buttons)] = joltage

	matrix = matrix.rref()[0]

	locked_button_presses = []
	button_guess_rows = []
	# Find all rows that lock in a value for button presses
	for i in range(matrix.shape[0]):
		row = matrix.row(i)
		non_zero_indices = [index for index, value in enumerate(row[:-1]) if value != 0]
		# Rows that lock in a button are rows where there is only one non-zero value,
		# or rows where the sum of several button presses is zero
		if len(non_zero_indices) == 1 or (row[-1] == 0 and (all(x <= 0 for x in row) or all(x >= 0 for x in row))):
			button_presses = row[-1]
			# If the number of button presses is negative or not an integer, this solution is impossible
			if button_presses < 0 or int(button_presses) != button_presses:
				return None
			
			for button_number in non_zero_indices:
				locked_button_presses.append((button_number, int(button_presses)))
			continue

		# Else, if all values in the row are non-negative, this provides a maximum limit to how many
		# times the buttons on that row can be pressed
		if not all(x == 0 for x in row):
			button_guess_rows.append(row)

		# If a row of all zeros is equal to a non-zero value, we have been given an impossible set of equations
		if all(x == 0 for x in row[:-1]) and row[-1] != 0:
			return None

	# If all buttons are locked, return the number of presses
	number_of_locked_presses = sum(t[1] for t in locked_button_presses)
	if len(locked_button_presses) == len(buttons):
		return number_of_locked_presses
		
	# Sort the button guess rows according to which we rather start guessing at. We prefer rows with only positive values, and
	# rows with fewer buttons, and then with lower maximum values
	button_guess_rows.sort(key=lambda row: row[-1])
	button_guess_rows.sort(key=lambda row: sum(1 for x in row if x != 0))
	button_guess_rows.sort(key=lambda row: 0 if all(x >= 0 for x in row) else 1)

	# Take the first button guess row, guess at as many button presses as necessary, lock the remaining button, call this function recursively
	# on the result

	minimum_presses = None
	
	guess_row = button_guess_rows[0]
	buttons_affected = [(index, x) for index, x in enumerate(guess_row[:-1]) if x != 0]
	guesses = [0 for i in range(len(buttons_affected) - 1)]
	max_guesses = [min(joltages[index] for index in buttons[button[0]]) for button in buttons_affected[:-1]]

	while guesses[0] <= max_guesses[0]:
		# Lock in all button presses
		final_button_presses = (guess_row[-1] - sum(buttons_affected[index][1] * guess for index, guess in enumerate(guesses))) / buttons_affected[-1][1]
		used_buttons = [button[0] for button in locked_button_presses] + [button[0] for button in buttons_affected]
		used_button_presses = [button[1] for button in locked_button_presses] + [guess for guess in guesses] + [int(final_button_presses)]

		guesses[-1] += 1
		for i in range(len(guesses)-2, -1, -1):
			if guesses[i+1] > max_guesses[i+1]:
				guesses[i+1] = 0
				guesses[i] += 1
		
		if final_button_presses != int(final_button_presses) or final_button_presses < 0:
			continue

		# Calculate a new set of buttons and joltages, and solve this recursively
		buttons_recursive = [button for index, button in enumerate(buttons) if index not in used_buttons]
		joltages_recursive = [joltage for joltage in joltages]
		for index, button_index in enumerate(used_buttons):
			button_presses = used_button_presses[index]

			for joltage_number in buttons[button_index]:
				joltages_recursive[joltage_number] -= button_presses

		if any(joltage < 0 for joltage in joltages_recursive):
			continue

		total_presses = None
		if len(buttons_recursive) == 0:
			if all(joltage == 0 for joltage in joltages_recursive):
				total_presses = sum(used_button_presses)
		else:
			recurse_presses = recursive_minimize(buttons_recursive, joltages_recursive)
			if recurse_presses is not None:
				total_presses = sum(used_button_presses) + recurse_presses

		if total_presses is not None and (minimum_presses is None or total_presses < minimum_presses):
			minimum_presses = total_presses

	return minimum_presses
		

fewest_presses = []
# Try to solve part two of problem by representing it as set of linear equations and finding matrices of reduced row echelon form
for index, (_, buttons, joltages) in enumerate(machines):
	print(f'\rWorking on index {index} of {len(machines)}', end='')
	fewest_presses.append(recursive_minimize(buttons, joltages))
print(f'\rThe fewest possible presses to set all the machines to the right joltages is {sum(fewest_presses)}')

In [ ]:
from collections import defaultdict, deque

reactors_input = '''aaa: you hhh
you: bbb ccc
bbb: ddd eee
ccc: ddd eee fff
ddd: ggg
eee: out
fff: out
ggg: out
hhh: ccc fff iii
iii: out'''.split('\n')
reactors_input = open('input11.txt', 'r').read().strip().split('\n')

connections = defaultdict(lambda: list())
backwards = defaultdict(lambda: list())

for line in reactors_input:
	source, sinks = line.split(':')
	for sink in sinks.strip().split(' '):
		connections[source].append(sink)
		backwards[sink].append(source)

ends = {}
for end in set(backwards.keys()).difference(set(connections.keys())):
	if end not in ends:
		ends[end] = {0: 0, 1:0, 2: 0, 3: 0}
ends['out'] = {0: 1, 1: 0, 2: 0, 3: 0}

def find_explorable(connections, ends):
	return next((source for source in connections if source not in ends and all(sink in ends for sink in connections[source])), None)
	
while (current := find_explorable(connections, ends)) is not None:
	ways_out = {0: 0, 1: 0, 2: 0, 3: 0}
	for connection in connections[current]:
		for key in ends[connection]:
			if ends[connection][key] == 0:
				continue

			addition = 0
			if current == 'dac':
				addition = 1
			elif current == 'fft':
				addition = 2
			ways_out[key + addition] += ends[connection][key]
	ends[current] = ways_out

print(f'There are {sum(ends["you"][key] for key in ends["you"])} different ways from you to finish')
print(f'There are {ends["svr"][3]} ways from svr to end via the dac and fft nodes')

In [ ]:
shape_data = '''0:
###
##.
##.

1:
###
##.
.##

2:
.##
###
##.

3:
##.
###
##.

4:
###
#..
###

5:
###
.#.
###

4x4: 0 0 0 0 2 0
12x5: 1 0 1 0 2 2
12x5: 1 0 1 0 3 2'''.split('\n\n')
shape_data = open('input12.txt', 'r').read().strip().split('\n\n')

shapes = []
shape_areas = []
for shape in shape_data[:-1]:
	area = 0
	shape_coordinates = set()
	for y, line in enumerate(shape.split('\n')[1:]):
		for x, c in enumerate(line):
			if c == '#':
				shape_coordinates.add((x, y))
				area += 1
	shape_coordinates = frozenset(shape_coordinates)
	shape_areas.append(area)
	rotated_shapes = set()
	rotated_shapes.add(shape_coordinates)

	for i in range(3):
		shape_coordinates = frozenset((2-coordinate[1], coordinate[0]) for coordinate in shape_coordinates)
		rotated_shapes.add(shape_coordinates)

	# Flip the shapes
	rotated_and_flipped_shapes = set()
	for rotated_shape in rotated_shapes:
		flipped_shape = frozenset((2 - x, y) for x, y in rotated_shape)
		rotated_and_flipped_shapes.add(rotated_shape)
		rotated_and_flipped_shapes.add(flipped_shape)

	shapes.append(frozenset(rotated_and_flipped_shapes))

can_fit = []
unknowns = []
for index, line in enumerate(shape_data[-1].split('\n')):
	rectangle, number_of_boxes = line.split(':')
	width, height = (int(x) for x in rectangle.split('x'))
	numbers_of_boxes = [int(x) for x in number_of_boxes.strip().split(' ')]

	# First idiot test
	minimal_total_area = sum(area * number for area, number in zip(shape_areas, numbers_of_boxes))
	if minimal_total_area > width*height:
		continue

	# Second idiot test
	maximal_total_area = sum(9 * number for number in numbers_of_boxes)
	area_to_fit_in = (width // 3) * 3 * (height // 3) * 3
	if maximal_total_area <= area_to_fit_in:
		can_fit.append(index)
		continue

	unknown.append(index)

# Turns out, this is a trick question!
# It is always obvious whether the gifts will fit or not!
if len(unknowns) != 0:
	print(f'This problem is very hard!')
else:
	print(f'There are {len(can_fit)} regions that can fit all the presents')